In [54]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [55]:
total_dict = {}
county_dict = {}
state_dict = {}
illinois_dict = {}
dph_illinois_dict = {}
who_world_dict = {}

In [56]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

1. Read County Data

In [57]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [58]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",78.0,5.0,2.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",32.0,0.0,4.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1042.0,14.0,113.0,1.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2.0,0.0,0.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",18.0,0.0,0.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [59]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [60]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))



1.1 Get Constant Classes from today's data

In [61]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [62]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [63]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [64]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '2.0,9.0,26.0,72.0,257.00000000000045,198710.0',
    'k': 6},
   'FisherJenks': {'bins': '2934.0,10427.0,20129.0,39573.0,64691.0,198710.0',
    'k': 6},
   'NaturalBreaks': {'bins': '2989.0,10427.0,20129.0,39573.0,64691.0,198710.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '1.0986122886681098,2.302585092994046,3.295836866004329,4.290459441148391,5.552959584921619,12.199606786846562',
    'k': 6},
   'FisherJenks': {'bins': '0.6931471805599453,2.3978952727983707,3.7612001156935624,5.19295685089021,7.05617528410041,12.199606786846562',
    'k': 6},
   'NaturalBreaks': {'bins': '0.6931471805599453,2.3978952727983707,3.7612001156935624,5.170483995038151,7.044032897274685,12.199606786846562',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,2.0,11.0,20376.0', 'k': 6},
   'FisherJenks': {'bins': '176.0,597.0,1160.0,1913.0,2980.0,20376.0', 'k': 6},
   'NaturalBreaks': {'bins': '176.0,597.0,1160.0,1913.0,2980.0,20376.0',
    'k':

1.2 Create Dynamic classes

In [65]:
# case_ts = counties_gdf['cases_ts'].apply(lambda x: x.split(','))
# death_ts = counties_gdf['deaths_ts'].apply(lambda x: x.split(','))

In [66]:
# length = len(case_ts[1])

In [67]:
# new_case_dict = {}
# for i in range(1,length):
#     q6 = mc.Quantiles(case_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

In [68]:
# new_death_dict = {}
# for i in range(1,length):
#     d6 = mc.Quantiles(death_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_death_dict[i] = ','.join(d6.bins.astype(int).astype(str))

In [69]:
# dynamic_county_dict['new_case'] = new_case_dict
# dynamic_county_dict['new_death'] = new_death_dict


2.Read State Data

In [70]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [71]:
state_gdf.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1514.0,12.0,68.0,0.0,2020-01-21,2020-05-19,day,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,2020-03-01,2020-03-06,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",46936.0,502.0,2051.0,55.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",98298.0,1520.0,4399.0,142.0,2020-01-21,2020-05-19,day,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",17038.0,657.0,757.0,17.0,2020-01-21,2020-05-19,day,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",41664.0,1782.0,2081.0,58.0,2020-01-21,2020-05-19,day,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [72]:
state_gdf.iloc[34]

NAME                                                  Virgin Islands
population                                                         0
fips                                                              78
dt_first_case                                             2020-03-14
dt_first_death                                            2020-04-06
cases_ts           0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....
deaths_ts          0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....
today_case                                                        69
today_new_case                                                     0
today_death                                                        6
today_new_death                                                    0
dt_start                                                  2020-01-21
dt_end                                                    2020-05-19
dt_unit                                                          day
geometry           (POLYGON ((-64.

In [73]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [74]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [75]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))



In [76]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [77]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

3. Read Illinois County Data

In [78]:
illinois_gdf = gpd.read_file('illinois/nyt_illinois_counties_data.geojson')

In [79]:
illinois_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Clark,Illinois,15836,17023,2020-03-30,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",9.0,0.0,0.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-88.01421 39.48076, -87.96018 ..."
1,Kendall,Illinois,124626,17093,2020-03-18,2020-03-29,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",602.0,23.0,22.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-88.60193 41.71956, -88.40450 ..."
2,Brown,Illinois,6675,17009,2020-05-01,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,-1.0,0.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-90.91347 40.10446, -90.69582 ..."
3,Warren,Illinois,17338,17187,2020-04-11,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",110.0,1.0,0.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-90.78519 41.06875, -90.43942 ..."
4,Union,Illinois,17127,17181,2020-04-15,2020-05-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",122.0,0.0,4.0,0.0,2020-01-21,2020-05-19,day,"MULTIPOLYGON (((-89.50179 37.55890, -89.49775 ..."


In [80]:
illinois_case = illinois_gdf['today_case']
illinois_death = illinois_gdf['today_death']
illinois_case_per_100k_capita = illinois_gdf['today_case']/(illinois_gdf['population']/100000)
illinois_death_per_100k_capita = illinois_gdf['today_death']/(illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [81]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [82]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))



In [83]:
illinois_dict['case'] = {}
illinois_dict['death'] = {}
illinois_dict['case_per_100k_capita'] = {}
illinois_dict['death_per_100k_capita'] = {}

In [84]:
illinois_dict['case']['nolog'] = create_dict(illinois_case)
illinois_dict['case']['log'] = create_dict(illinois_log_case)
illinois_dict['death']['nolog'] = create_dict(illinois_death)
illinois_dict['death']['log'] = create_dict(illinois_log_death)
illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

4. Read DPH Illinois County Data

In [85]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [86]:
dph_illinois_gdf.head(5)

,id,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,dt_start,dt_end,dt_unit,population,geometry
0,McHenry,McHenry,"2.0,4.0,6.0,8.0,11.0,12.0,12.0,14.0,19.0,27.0,...",2020-03-17,1255.0,31.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2....",2020-03-26,66.0,2.0,2020-03-17,2020-05-20,day,1,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
1,Boone,Boone,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-05,324.0,10.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-08,14.0,0.0,2020-03-17,2020-05-20,day,1,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
2,Ogle,Ogle,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-31,174.0,1.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-15,2.0,0.0,2020-03-17,2020-05-20,day,1,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
3,Will,Will,"2.0,3.0,9.0,10.0,12.0,21.0,21.0,28.0,40.0,67.0...",2020-03-17,4694.0,134.0,"0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4....",2020-03-20,247.0,3.0,2020-03-17,2020-05-20,day,1,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
4,LaSalle,LaSalle,"0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3....",2020-03-19,130.0,9.0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-29,8.0,0.0,2020-03-17,2020-05-20,day,1,"POLYGON ((-88.93885 41.62837, -88.93891 41.628..."


In [87]:
dph_illinois_case = dph_illinois_gdf['today_case']

#illinois_case_per_100k_capita = illinois_gdf['today_case']/(illinois_gdf['population']/100000)
#illinois_death_per_100k_capita = illinois_gdf['today_death']/(illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [88]:
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [89]:
dph_illinois_log_case = dph_illinois_case.apply(lambda x: np.log(x + 1))
# illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
# illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
# illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))



In [90]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [91]:
dph_illinois_dict['case']['nolog'] = create_dict(dph_illinois_case)
dph_illinois_dict['case']['log'] = create_dict(dph_illinois_log_case)
# dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
# dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
# dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
# dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

6. Read DPH County Test (Static) and DPH Zipcode Case (Static)


In [92]:
dph_illinois_test_gdf = gpd.read_file('illinois/dph_county_static_data.geojson')
dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [93]:
dph_zipcode_gdf

,id,confirmed_cases,total_tested,geometry
0,62359,0.0,0.0,"POLYGON ((-91.18290 40.02688, -91.17656 40.020..."
1,62376,0.0,0.0,"POLYGON ((-91.35721 40.17576, -91.35600 40.151..."
2,62312,0.0,0.0,"POLYGON ((-90.97797 39.78881, -90.97787 39.772..."
3,62365,0.0,0.0,"POLYGON ((-91.16584 39.83701, -91.14703 39.837..."
4,62246,7.0,163.0,"POLYGON ((-89.56254 38.91811, -89.56794 38.923..."
...,...,...,...,...
1379,60484,101.0,505.0,"POLYGON ((-87.70969 41.45644, -87.70949 41.456..."
1380,62948,8.0,324.0,"MULTIPOLYGON (((-89.02159 37.74516, -89.03561 ..."
1381,62921,0.0,0.0,"POLYGON ((-89.09155 37.79258, -89.08896 37.807..."
1382,62915,0.0,0.0,"POLYGON ((-89.11994 37.78039, -89.12263 37.781..."


In [94]:
dph_illinois_tested = dph_illinois_test_gdf['total_tested']
dph_zipcode_tested = dph_zipcode_gdf['total_tested']
dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [95]:
dph_illinois_log_tested = dph_illinois_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [96]:
dph_illinois_dict['tested'] = {}
dph_illinois_dict['zipcode_case'] = {}
dph_illinois_dict['zipcode_tested'] = {}

In [97]:
dph_illinois_dict['tested']['nolog'] = create_dict(dph_illinois_tested)
dph_illinois_dict['tested']['log'] = create_dict(dph_illinois_log_tested)
dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)




C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

7. Read Worldwide Data

In [98]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

In [99]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']

In [100]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))

In [101]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}

In [102]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)

Summary and ouput


In [103]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['illinois'] = illinois_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_dict, json_file)
print('done')

done
